In [101]:
%matplotlib inline
#%load_ext autoreload
#%autoreload 2
%reload_ext autoreload
import numpy as np
import matplotlib.pyplot as plt
import math, sys, os
from numpy.random import randn


# setup pyspark for IPython_notebooks
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.8.2.1-src.zip'))
execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by <module> at <ipython-input-1-1cb5f5e6034d>:13 

# Top-level RDD methods

In [34]:
RDD = sc.parallelize(range(1000))

In [85]:
print "collect: "
print RDD.collect()[:15]

print ""
print "take(15): "
print RDD.take(15)

print ""
print "takeSample(False,15)"
print RDD.takeSample(False,15)

print ""
print "isEmpty: "
print RDD.isEmpty()

print ""
print "count: "
print RDD.count()

print ""
print "countApprox(timeout in seconds, confidence interval): "
print RDD.countApprox(1,0.95)

print ""
print "min, max: "
print RDD.min()
print RDD.max()

print ""
print "sum"
print RDD.sum()

print ""
print "mean: "
print RDD.mean()

print ""
print "meanApprox(timeout in seconds, confidence=0.95): "
print RDD.meanApprox(1, confidence=0.95)

print ""
print "stdev"
print RDD.stdev()

print ""
print "variance"
print RDD.variance()


print ""
print "histogram(10)"
bins, counts = RDD.histogram(10)
print "bins: "
print bins
print "counts: "
print counts

collect: 
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

take(15): 
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

takeSample(False,15)
[651, 476, 772, 844, 743, 170, 576, 768, 187, 623, 841, 23, 145, 401, 128]

isEmpty: 
False

count: 
1000

countApprox(timeout in seconds, confidence interval): 
1000

min, max: 
0
999

sum
499500

mean: 
499.5

meanApprox(timeout in seconds, confidence=0.95): 
499.5

stdev
288.674990257

variance
83333.25

histogram(10)
bins: 
[0.0, 99.9, 199.8, 299.70000000000005, 399.6, 499.5, 599.4000000000001, 699.3000000000001, 799.2, 899.1, 999]
counts: 
[100, 100, 100, 100, 100, 100, 100, 100, 100, 100]


# Higher-Order Functions

In [69]:
# fitler
print "RDD.filter( lambda x: x > 990).collect()"
RDD.filter( lambda x: x > 990).collect()

print ""
print "RDD.map(lambda x: x % 4).take(10)"
print RDD.map(lambda x: x % 4).take(10)

print ""
print "RDD.reduce(lambda a, b: min(a, b))"
print RDD.reduce(lambda a, b: min(a, b))
print "RDD.reduce(lambda a, b: max(a, b))"
print RDD.reduce(lambda a, b: max(a, b))

print ""
print "RDD.filter(lambda x: x == 2).collect()"
print RDD.filter(lambda x: x == 4).collect()
print "RDD.flatMap(lambda x: [x] * (x % 5)).filter(lambda x: x == 4).collect()"
print RDD.flatMap(lambda x: [x] * (x % 5)).filter(lambda x: x == 4).collect()

RDD.filter( lambda x: x > 990).collect()

RDD.map(lambda x: x % 4).take(10)
[0, 1, 2, 3, 0, 1, 2, 3, 0, 1]

RDD.reduce(lambda a, b: min(a, b))
0
RDD.reduce(lambda a, b: max(a, b))
999

RDD.filter(lambda x: x == 2).collect()
[4]
RDD.flatMap(lambda x: [x] * (x % 5)).filter(lambda x: x == 4).collect()
[4, 4, 4, 4]


# RDD interactions

In [90]:
import string
alpha = string.ascii_lowercase
num_RDD = sc.parallelize(range(26))
alpha_RDD = sc.parallelize(alpha)

# zipping two rdds
alpha_num_RDD = alpha_RDD.zip(num_RDD)

print alpha_RDD.collect()
print ""
print num_RDD.collect()
print ""
print alpha_num_RDD.collect()
print ""
print "RDD.collect() == zip(alpha,range(26))"
print alpha_num_RDD.collect() == zip(alpha,range(26))

# cartesian product RDD
# for every item in A pair with every item in B.
# For RDDs of size N and M produces a new RDD of size N*M
print ""
cart_RDD = alpha_RDD.cartesian(num_RDD)
print "cartesian RDD size: "
print cart_RDD.count()





['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]

[('a', 0), ('b', 1), ('c', 2), ('d', 3), ('e', 4), ('f', 5), ('g', 6), ('h', 7), ('i', 8), ('j', 9), ('k', 10), ('l', 11), ('m', 12), ('n', 13), ('o', 14), ('p', 15), ('q', 16), ('r', 17), ('s', 18), ('t', 19), ('u', 20), ('v', 21), ('w', 22), ('x', 23), ('y', 24), ('z', 25)]

RDD.collect() == zip(alpha,range(26))
True

cartesian RDD size: 
676

subtract (set difference)
[('x', 1), ('y', 4), ('y', 5)]


# Set operations

In [100]:
print "A = sc.parallelize([('x', 1), ('y', 4), ('y', 5), ('x', 2)])"
print "sc.parallelize([('x', 2), ('y', 5), ('z', 6)])"
A = sc.parallelize([("x", 1), ("y", 4), ("y", 5), ("x", 2)])
B = sc.parallelize([("x", 2), ("y", 5), ("z", 6)])

print ""
print "A.union(B).collect()"
print sorted(A.union(B).collect())

print ""
print "union (distict)"
print "A.union(B).distinct().collect()"
print sorted(A.union(B).distinct().collect())

print ""
print "intersection"
print "A.intersection(B).collect()"
print sorted(A.intersection(B).collect())

print ""
print "subtract (set difference)"
print "A.subtract(B).collect()"
print sorted(A.subtract(B).collect())

A = sc.parallelize([('x', 1), ('y', 4), ('y', 5), ('x', 2)])
sc.parallelize([('x', 2), ('y', 5), ('z', 6)])

A.union(B).collect()
[('x', 1), ('x', 2), ('x', 2), ('y', 4), ('y', 5), ('y', 5), ('z', 6)]

union (distict)
A.union(B).distinct().collect()
[('x', 1), ('x', 2), ('y', 4), ('y', 5), ('z', 6)]

intersection
A.intersection(B).collect()
[('x', 2), ('y', 5)]

subtract (set difference)
A.subtract(B).collect()
[('x', 1), ('y', 4)]


# Key-Value basics

In [86]:
print "keyBy"
print "RDD.keyBy(lambda x: x % 5).take(10)"
print RDD.keyBy(lambda x: x % 5).take(10)

print ""
print "keys"
print "RDD.keyBy(lambda x: x % 5).keys().distinct().count()"
print RDD.keyBy(lambda x: x % 5).keys().distinct().count()

print ""
print "values"
print "RDD.keyBy(lambda x: x % 5).values().distinct().count()"
print RDD.keyBy(lambda x: x % 5).values().distinct().count()

print ""
print "collectAsMap "
print alpha_num_RDD.collectAsMap()

print ""
print "countByKey"
print dict(cart_RDD.countByKey())

print ""
print "reduceByKey"
print "RDD.keyBy(lambda x: x % 5).reduceByKey(lambda a, b: max(a,b)).collectAsMap()"
print RDD.keyBy(lambda x: x % 5).reduceByKey(lambda a, b: max(a,b)).collectAsMap()

keyBy
RDD.keyBy(lambda x: x % 5).take(10)
[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4), (0, 5), (1, 6), (2, 7), (3, 8), (4, 9)]

keys
RDD.keyBy(lambda x: x % 5).keys().distinct().count()
5

values
RDD.keyBy(lambda x: x % 5).values().distinct().count()
1000

collectAsMap: 
{'a': 0, 'c': 2, 'b': 1, 'e': 4, 'd': 3, 'g': 6, 'f': 5, 'i': 8, 'h': 7, 'k': 10, 'j': 9, 'm': 12, 'l': 11, 'o': 14, 'n': 13, 'q': 16, 'p': 15, 's': 18, 'r': 17, 'u': 20, 't': 19, 'w': 22, 'v': 21, 'y': 24, 'x': 23, 'z': 25}

countByKey
{'a': 26, 'c': 26, 'b': 26, 'e': 26, 'd': 26, 'g': 26, 'f': 26, 'i': 26, 'h': 26, 'k': 26, 'j': 26, 'm': 26, 'l': 26, 'o': 26, 'n': 26, 'q': 26, 'p': 26, 's': 26, 'r': 26, 'u': 26, 't': 26, 'w': 26, 'v': 26, 'y': 26, 'x': 26, 'z': 26}

reduceByKey
RDD.keyBy(lambda x: x % 5).reduceByKey(lambda a, b: max(a,b)).collectAsMap()
{0: 995, 1: 996, 2: 997, 3: 998, 4: 999}
